# Scraping des données Allociné

Ce notebook récupère et analyse des informations sur les acteurs d’un film depuis Allociné.  
Les étapes principales incluent :
1. Extraction des liens des acteurs depuis la page de casting.
2. Scraping des pages individuelles pour récupérer les dates de naissance.
3. Conversion des dates et calcul de l’âge des acteurs (année de référence : 2014).

Les bibliothèques utilisées : **BeautifulSoup**, **Requests**, **Pandas**, **re**, **Datetime**.  
Le résultat final est un tableau structuré avec noms, liens, dates de naissance et âges.

In [1]:
# Importation des bibliothèques nécessaires
from bs4 import BeautifulSoup  # Pour extraire les données du HTML
import requests  # Pour envoyer des requêtes HTTP et récupérer le contenu des pages
import pandas as pd  # Pour organiser les données sous forme de tableau
import re  # Pour la gestion des expressions régulières (non utilisé ici mais importé)
from datetime import datetime  # Pour manipuler des dates (non utilisé ici mais importé)

In [2]:
# URL de la page du film sur Allociné
url = "https://www.allocine.fr/film/fichefilm-114782/casting/"

# Envoi de la requête et récupération de la page HTML
html = requests.get(url)

# Analyse du contenu HTML avec BeautifulSoup
soup = BeautifulSoup(html.text, 'html.parser')

# Extraction de la section contenant les acteurs
soup_actors = soup.find('section', {"class": "section casting-actor"})

# Récupération des liens des acteurs
links = soup_actors.find_all('a')

In [3]:
# Affichage d'un exemple de lien
print(links[0].text)  # Affiche le nom de l'acteur
print(links[0]['href'])  # Affiche le lien vers la page de l'acteur

Matthew McConaughey
/personne/fichepersonne_gen_cpersonne=19334.html


In [4]:
# Création d'un DataFrame avec les noms et les liens
df_url = pd.DataFrame({
    "actor": [i.text for i in links],
    "href": [i["href"] for i in links]
})

# Affichage du DataFrame
display(df_url)

,actor,href
0,Matthew McConaughey,/personne/fichepersonne_gen_cpersonne=19334.html
1,Anne Hathaway,/personne/fichepersonne_gen_cpersonne=65719.html
2,Michael Caine,/personne/fichepersonne_gen_cpersonne=2535.html
3,John Lithgow,/personne/fichepersonne_gen_cpersonne=5293.html
4,Jessica Chastain,/personne/fichepersonne_gen_cpersonne=117304.html
5,Casey Affleck,/personne/fichepersonne_gen_cpersonne=1196.html
6,Mackenzie Foy,/personne/fichepersonne_gen_cpersonne=487728.html
7,Wes Bentley,/personne/fichepersonne_gen_cpersonne=35000.html
8,David Gyasi,/personne/fichepersonne_gen_cpersonne=205411.html
9,Timothée Chalamet,/personne/fichepersonne_gen_cpersonne=525421.html


In [5]:
# Construction de l'URL complète vers la page de l'acteur
url = "https://www.allocine.fr" + "/personne/fichepersonne_gen_cpersonne=525421.html"

# Affichage de l'URL générée pour vérification
print(url)

https://www.allocine.fr/personne/fichepersonne_gen_cpersonne=525421.html


In [6]:
# Envoi d'une requête HTTP pour récupérer le contenu de la page de l'acteur
html = requests.get(url)

In [7]:
# Analyse du contenu HTML avec BeautifulSoup
soup = BeautifulSoup(html.text, 'html.parser')

# Recherche de la section contenant les informations détaillées sur l'acteur
detail = soup.find("div", {"class": "meta-body"})

# Affichage de la section pour vérifier les données extraites
print(detail)

<div class="meta-body">
<div class="meta-body-item">
<span class="light">Métiers </span>
<strong>
<span class="ACrL3BACrlcnNvbm5lL2ZpY2hlcGVyc29ubmUtNTI1NDIxL2ZpbG1vZ3JhcGhpZS8=#actor dark-grey-link">
                                Acteur
                            </span>,                                                 </strong>
<strong>
<span class="ACrL3BACrlcnNvbm5lL2ZpY2hlcGVyc29ubmUtNTI1NDIxL2ZpbG1vZ3JhcGhpZS8=#production dark-grey-link">
                                Producteur
                            </span>,                                                 </strong>
<strong>
<span class="ACrL3BACrlcnNvbm5lL2ZpY2hlcGVyc29ubmUtNTI1NDIxL2ZpbG1vZ3JhcGhpZS8=#music dark-grey-link">
                                Interprète (chansons du film)
                            </span> </strong>
</div>
<div class="meta-body-item">
<span class="light">Nom de naissance </span>
<h2 class="item">Timothée Hal Chalamet</h2>
</div>
<div class="meta-body-item">
<span class="light">Nationali

In [8]:
# Extraction de tous les textes des balises <span> dans la section "meta-body"
infos = [detail.find_all("span")[i].text for i in range(len(detail.find_all("span")))]

# Affichage des informations collectées pour validation
print(infos)

['Métiers ', '\n                                Acteur\n                            ', '\n                                Producteur\n                            ', '\n                                Interprète (chansons du film)\n                            ', 'Nom de naissance ', 'Nationalités ', 'Naissance ', '\n                                    27 décembre 1995\n    \n                    ', 'Age ']


In [9]:
# Recherche d'une date spécifique (format : "JJ mois AAAA") dans les textes extraits
print(re.findall("\d{2}\s\w+\s\d{4}", [detail.find_all("span")[i].text for i in range(len(detail.find_all("span")))][7]))

['27 décembre 1995']


In [10]:
# Initialisation d'une liste pour stocker les dates extraites
date = []

# Parcours des informations et ajout des dates extraites à la liste
for info in infos:
    if re.findall("\d{2}\s\w+\s\d{4}", info):
        # Ajout de la première date trouvée
        date.append(re.findall("\d{2}\s\w+\s\d{4}", info)[0])

# Affichage de la première date trouvée
print(date[0])

27 décembre 1995


In [11]:
# Fonction pour extraire la date de naissance depuis une page individuelle d'acteur
def crawl(url_short):
    # Construction de l'URL complète
    url = "https://www.allocine.fr" + url_short
    
    # Envoi d'une requête pour récupérer le contenu de la page
    html = requests.get(url)
    
    # Parsing du contenu HTML avec BeautifulSoup
    soup = BeautifulSoup(html.text, 'html.parser')
    
    # Recherche de la section contenant les informations sur l'acteur
    detail = soup.find("div", {"class": "meta-body"})
    try:
        # Extraction de toutes les informations textuelles de la section
        infos = [detail.find_all("span")[i].text for i in range(len(detail.find_all("span")))]
        
        # Initialisation d'une liste pour stocker les dates trouvées
        date = []
        
        # Recherche et ajout des dates au format "JJ mois AAAA" dans la liste
        for info in infos:
            if re.findall("\d+\s\w+\s\d{4}", info):
                date.append(re.findall("\d+\s\w+\s\d{4}", info)[0])
        
        # Retourne la première date trouvée (si disponible)
        return date[0]
    except:
        # Gestion des erreurs (pass si aucune date n'est trouvée)
        pass

In [12]:
# Application de la fonction 'crawl' pour extraire les dates de naissance
df_url['date_de_naissance'] = df_url.href.apply(crawl)

In [13]:
def convertir_date(date_str):
    # Vérifie si la valeur est nulle
    if pd.isna(date_str):
        return None

    # Dictionnaire pour convertir les mois en français
    mois_francais = {
        'janvier': 'January',
        'février': 'February',
        'mars': 'March',
        'avril': 'April',
        'mai': 'May',
        'juin': 'June',
        'juillet': 'July',
        'août': 'August',
        'septembre': 'September',
        'octobre': 'October',
        'novembre': 'November',
        'décembre': 'December'
    }

    # Conversion du mois français en anglais
    for mois_fr, mois_en in mois_francais.items():
        if mois_fr in date_str:
            date_str = date_str.replace(mois_fr, mois_en)

    # Conversion en objet datetime
    date_obj = datetime.strptime(date_str, "%d %B %Y")

    # Retourne la date au format souhaité
    return date_obj.strftime("%d/%m/%Y")

In [14]:
# Applique la fonction 'convertir_date' pour convertir les dates en format standard
df_url['date_de_naissance'] = df_url.date_de_naissance.apply(convertir_date)

# Convertit les dates en objets datetime pour faciliter les calculs
df_url['date_de_naissance'] = pd.to_datetime(df_url['date_de_naissance'], format='%d/%m/%Y', errors='coerce')

In [15]:
# Calcule l'âge des acteurs en prenant 2014 comme année de référence
df_url['age'] = df_url.date_de_naissance.apply(lambda x: 2014 - x.year)

In [16]:
display(df_url)

,actor,href,date_de_naissance,age
0,Matthew McConaughey,/personne/fichepersonne_gen_cpersonne=19334.html,1969-11-04,45.0
1,Anne Hathaway,/personne/fichepersonne_gen_cpersonne=65719.html,1982-11-12,32.0
2,Michael Caine,/personne/fichepersonne_gen_cpersonne=2535.html,1933-03-14,81.0
3,John Lithgow,/personne/fichepersonne_gen_cpersonne=5293.html,1945-10-19,69.0
4,Jessica Chastain,/personne/fichepersonne_gen_cpersonne=117304.html,1977-03-24,37.0
5,Casey Affleck,/personne/fichepersonne_gen_cpersonne=1196.html,1975-08-12,39.0
6,Mackenzie Foy,/personne/fichepersonne_gen_cpersonne=487728.html,2000-11-10,14.0
7,Wes Bentley,/personne/fichepersonne_gen_cpersonne=35000.html,1978-09-04,36.0
8,David Gyasi,/personne/fichepersonne_gen_cpersonne=205411.html,1980-01-02,34.0
9,Timothée Chalamet,/personne/fichepersonne_gen_cpersonne=525421.html,1995-12-27,19.0


In [19]:
# Calcule la moyenne de l'âge des acteurs dans le DataFrame
print(round(df_url.age.mean(), 2))

44.06
